# BEST DEAL WITH FLIGHTS - CLUSTERING

In [230]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

STEP1: Data collection has been done by web scrapping the KAYAK website to get the details about the flights available between certain places. The collected data saved in CSV file which has the important features like Airline, Source, Destination, Duraion, Departure time, Arrival time, Total stops, Layover city,Layover time, Checkin and cabin bags , Price

In [209]:
#IMPORTING THE csv FILE AS DATAFRAME
df = pd.read_csv('NYC_DXB_JAN.csv',index_col='Unnamed: 0')

In [152]:
#printing the DF
df.tail()

,Date,Airline,Duration,Source,Destination,Departure Time,Arrival Time,Layover Time,Layover Cities,Total stops,Carry bags Count,Checkin bags Count,Price
2740,2024-01-31,British Airways,19h 55m,New York,Dubai,6:50 pm,11:45 pm,6h 00m,London Heathrow,1,1.0,1.0,"4,067"
2741,2024-01-31,British Airways,20h 15m,New York,Dubai,6:30 pm,11:45 pm,6h 20m,London Heathrow,1,1.0,1.0,"4,067"
2742,2024-01-31,"Porter Airlines, Emirates",18h 50m,New York,Dubai,7:50 am,11:40 am,4h 25m,Airport change YTZ-YYZ,1,1.0,2.0,"7,549"
2743,2024-01-31,Qatar Airways,19h 50m,New York,Dubai,11:00 pm,3:50 am,"2h 55m,2h 10m","Barcelona-El Prat,Doha Hamad Intl",2,1.0,2.0,"7,483"
2744,2024-01-31,Emirates,18h 55m,New York,Dubai,3:15 pm,7:10 pm,2h 47m,Chicago O'Hare Intl,1,1.0,2.0,"9,412"


In [140]:
#shape shows the amount of rows and columns
df.shape

(2745, 13)

In [141]:
df.describe()

,Total stops,Carry bags Count,Checkin bags Count
count,2745.000000,2744.000000,2744.000000
mean,1.288160,0.965379,1.537172
std,0.579366,0.182851,0.700557
min,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,1.000000,2.000000
75%,2.000000,1.000000,2.000000
max,3.000000,1.000000,2.000000


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2745 entries, 0 to 2744
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                2745 non-null   object 
 1   Airline             2745 non-null   object 
 2   Duration            2035 non-null   object 
 3   Source              2745 non-null   object 
 4   Destination         2745 non-null   object 
 5   Departure Time      2745 non-null   object 
 6   Arrival Time        2745 non-null   object 
 7   Layover Time        2587 non-null   object 
 8   Layover Cities      2587 non-null   object 
 9   Total stops         2745 non-null   int64  
 10  Carry bags Count    2744 non-null   float64
 11  Checkin bags Count  2744 non-null   float64
 12  Price               2745 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 300.2+ KB


# Step2: DATA WRAGLING

Since the data types of certain features are not in proper way like we price feature as object instead of int and Data is also object but is hsould be datatime etc

In [210]:
#caling function to change data type of date columns
def change_datatype(df):
    #changing the data type for below columns
    df['Date'] = pd.to_datetime(df['Date'])
    df['Departure Time'] = pd.to_datetime(df['Departure Time'])
#     df['Departure Time'] = df['Departure Time'].dt.time
    df['Arrival Time'] = pd.to_datetime(df['Arrival Time'])
#     df['Arrival Time'] = df['Arrival Time'].dt.time
    df['Price'] = df['Price'].str.replace(',', '', regex=True)  # Remove commas
    df['Price'] = pd.to_numeric(df['Price'])
#     df['Price'] = pd.to_numeric(df['Price'], errors='coerce', downcast='integer')
    return df

In [211]:
#Applying the above function on the df
change_datatype(df)
df.head()

,Date,Airline,Duration,Source,Destination,Departure Time,Arrival Time,Layover Time,Layover Cities,Total stops,Carry bags Count,Checkin bags Count,Price
0,2024-01-01,Emirates,12h 25m,New York,Dubai,2023-11-01 23:00:00,2023-11-01 20:25:00,NaN,NaN,0,1.0,2.0,1138
1,2024-01-01,EL AL,15h 35m,New York,Dubai,2023-11-01 17:00:00,2023-11-01 17:35:00,2h 15m,Tel Aviv Ben Gurion Intl,1,1.0,1.0,729
2,2024-01-01,Qatar Airways,23h 50m,New York,Dubai,2023-11-01 01:25:00,2023-11-01 10:15:00,10h 10m,Doha Hamad Intl,1,1.0,2.0,721
3,2024-01-01,Kuwait Airways,15h 55m,New York,Dubai,2023-11-01 17:00:00,2023-11-01 17:55:00,2h 00m,Kuwait City,1,1.0,2.0,787
4,2024-01-01,Emirates,12h 25m,New York,Dubai,2023-11-01 23:00:00,2023-11-01 20:25:00,NaN,NaN,0,1.0,2.0,1121


In [212]:
# Function to convert duration to minutes and layover time to total minutes of layover time
def convert_time_to_minutes(col):
    layover_times = col.split(',')  # Split by commas to get individual layover times
    total_minutes = 0
    for layover_time in layover_times:
        parts = layover_time.split()  # Split each layover time by whitespace
        for part in parts:
            if 'h' in part:
                total_minutes += int(part.strip('h')) * 60
            elif 'm' in part:
                total_minutes += int(part.strip('m'))
    return total_minutes


In [213]:
#Applying above function to duration and layover time cols
df['Duration'] = df['Duration'].astype(str) 
df['Duration'] = df['Duration'].apply(convert_time_to_minutes)

In [214]:
df['Layover Time'] = df['Layover Time'].astype(str)
df['Layover Time'] = df['Layover Time'].apply(convert_time_to_minutes)

In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2745 entries, 0 to 2744
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                2745 non-null   datetime64[ns]
 1   Airline             2745 non-null   object        
 2   Duration            2745 non-null   int64         
 3   Source              2745 non-null   object        
 4   Destination         2745 non-null   object        
 5   Departure Time      2745 non-null   datetime64[ns]
 6   Arrival Time        2745 non-null   datetime64[ns]
 7   Layover Time        2745 non-null   int64         
 8   Layover Cities      2587 non-null   object        
 9   Total stops         2745 non-null   int64         
 10  Carry bags Count    2744 non-null   float64       
 11  Checkin bags Count  2744 non-null   float64       
 12  Price               2745 non-null   int64         
dtypes: datetime64[ns](3), float64(2), int64(4), obje

In [216]:
df.tail()

,Date,Airline,Duration,Source,Destination,Departure Time,Arrival Time,Layover Time,Layover Cities,Total stops,Carry bags Count,Checkin bags Count,Price
2740,2024-01-31,British Airways,1195,New York,Dubai,2023-11-01 18:50:00,2023-11-01 23:45:00,360,London Heathrow,1,1.0,1.0,4067
2741,2024-01-31,British Airways,1215,New York,Dubai,2023-11-01 18:30:00,2023-11-01 23:45:00,380,London Heathrow,1,1.0,1.0,4067
2742,2024-01-31,"Porter Airlines, Emirates",1130,New York,Dubai,2023-11-01 07:50:00,2023-11-01 11:40:00,265,Airport change YTZ-YYZ,1,1.0,2.0,7549
2743,2024-01-31,Qatar Airways,1190,New York,Dubai,2023-11-01 23:00:00,2023-11-01 03:50:00,305,"Barcelona-El Prat,Doha Hamad Intl",2,1.0,2.0,7483
2744,2024-01-31,Emirates,1135,New York,Dubai,2023-11-01 15:15:00,2023-11-01 19:10:00,167,Chicago O'Hare Intl,1,1.0,2.0,9412


In [217]:
#Extrating Date,month,year and day of week from Date col
def extract_date_components(data, col):
    data[col + '_year'] = data[col].dt.year
    data[col + '_month'] = data[col].dt.month
    data[col + '_day'] = data[col].dt.day
    data[col + '_day_of_week'] = data[col].dt.dayofweek

    # Drop the original column
    data.drop(col, axis=1, inplace=True)

In [218]:
extract_date_components(df, 'Date')

In [219]:
#Extrating hours and minutes from Arrival and departure times cols
def extract_hour_and_minute(data, col):
    # Extract hour and minute
    data[col + '_hour'] = data[col].dt.hour
    data[col + '_minute'] = data[col].dt.minute

    # Drop the original column
    data.drop(col, axis=1, inplace=True)

In [220]:
extract_hour_and_minute(df,'Departure Time')
extract_hour_and_minute(df,'Arrival Time')

In [221]:
df.head()

,Airline,Duration,Source,Destination,Layover Time,Layover Cities,Total stops,Carry bags Count,Checkin bags Count,Price,Date_year,Date_month,Date_day,Date_day_of_week,Departure Time_hour,Departure Time_minute,Arrival Time_hour,Arrival Time_minute
0,Emirates,745,New York,Dubai,0,NaN,0,1.0,2.0,1138,2024,1,1,0,23,0,20,25
1,EL AL,935,New York,Dubai,135,Tel Aviv Ben Gurion Intl,1,1.0,1.0,729,2024,1,1,0,17,0,17,35
2,Qatar Airways,1430,New York,Dubai,610,Doha Hamad Intl,1,1.0,2.0,721,2024,1,1,0,1,25,10,15
3,Kuwait Airways,955,New York,Dubai,120,Kuwait City,1,1.0,2.0,787,2024,1,1,0,17,0,17,55
4,Emirates,745,New York,Dubai,0,NaN,0,1.0,2.0,1121,2024,1,1,0,23,0,20,25


# Missing values

In [222]:
df.isnull().sum()

Airline                    0
Duration                   0
Source                     0
Destination                0
Layover Time               0
Layover Cities           158
Total stops                0
Carry bags Count           1
Checkin bags Count         1
Price                      0
Date_year                  0
Date_month                 0
Date_day                   0
Date_day_of_week           0
Departure Time_hour        0
Departure Time_minute      0
Arrival Time_hour          0
Arrival Time_minute        0
dtype: int64

In [223]:
df.isnull().mean()

Airline                  0.000000
Duration                 0.000000
Source                   0.000000
Destination              0.000000
Layover Time             0.000000
Layover Cities           0.057559
Total stops              0.000000
Carry bags Count         0.000364
Checkin bags Count       0.000364
Price                    0.000000
Date_year                0.000000
Date_month               0.000000
Date_day                 0.000000
Date_day_of_week         0.000000
Departure Time_hour      0.000000
Departure Time_minute    0.000000
Arrival Time_hour        0.000000
Arrival Time_minute      0.000000
dtype: float64

In [239]:
# Create a mask to identify rows with any null values
null_mask = (df['Layover Cities'].isna()) & (df['Total stops'] != 0)

# Filter the DataFrame to show only rows with any null values
df_with_nulls = df[null_mask]
df_with_nulls.head()

,Duration,Layover Time,Layover Cities,Total stops,Carry bags Count,Checkin bags Count,Price,Date_year,Date_month,Date_day,...,Royal Jordanian,SAUDIA,SWISS,Singapore Airlines,TAP AIR PORTUGAL,Turkish Airlines,United Airlines,Vueling,New York,Dubai


It is observed that where ever the Layover cities value is NaN, Total stops is 0 which suggests that If there is no  stops then there will be no Layover cities. We decided to drop this col since it gives less impact to get best deal of flights, so is Data_year. As we have data with in 2024 so it is unique vlaue for the feature.

In [241]:
#Handling null values
def drop_cols(data,col):
    data.drop(col, axis=1, inplace=True)

In [245]:
drop_cols(df,'Layover Cities')
drop_cols(df,'Date_year')

In [231]:
def one_hot_encode_and_concat(df, column_name):
    df[column_name] = df[column_name].astype(str)
    df[column_name] = df[column_name].str.split(',')
    df[column_name] = df[column_name].apply(lambda x: [item.strip("[]''") for item in x])
    mlb = MultiLabelBinarizer()
    array_out = mlb.fit_transform(df[column_name])
    df_out = pd.DataFrame(data=array_out, columns=mlb.classes_)
    df_concatenated = pd.concat([df, df_out], axis=1)
    df_concatenated.drop(column_name, axis=1, inplace=True)  # Drop the original column
    return df_concatenated

# Example usage:
# df_concatenated = one_hot_encode_and_concat(df, 'Airline')
# print(df_concatenated)
df = one_hot_encode_and_concat(df, 'Airline')
df = one_hot_encode_and_concat(df, 'Source')
df = one_hot_encode_and_concat(df, 'Destination')

In [246]:
df.head()

,Duration,Layover Time,Total stops,Carry bags Count,Checkin bags Count,Price,Date_month,Date_day,Date_day_of_week,Departure Time_hour,...,Royal Jordanian,SAUDIA,SWISS,Singapore Airlines,TAP AIR PORTUGAL,Turkish Airlines,United Airlines,Vueling,New York,Dubai
0,745,0,0,1.0,2.0,1138,1,1,0,23,...,0,0,0,0,0,0,0,0,1,1
1,935,135,1,1.0,1.0,729,1,1,0,17,...,0,0,0,0,0,0,0,0,1,1
2,1430,610,1,1.0,2.0,721,1,1,0,1,...,0,0,0,0,0,0,0,0,1,1
3,955,120,1,1.0,2.0,787,1,1,0,17,...,0,0,0,0,0,0,0,0,1,1
4,745,0,0,1.0,2.0,1121,1,1,0,23,...,0,0,0,0,0,0,0,0,1,1


In [247]:
df.shape

(2745, 73)

# Outliers